In [1]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf

2024-04-20 16:26:31.404568: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-20 16:26:31.502337: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-20 16:26:31.914941: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-20 16:26:34.090554: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
## 1. Dataset das imagens
dataset_dir = os.path.join(os.getcwd(), 'imagens')
dataset_morango_dir = os.path.join(dataset_dir, 'morango')
dataset_pessego_dir = os.path.join(dataset_dir, 'pessego')
dataset_roma_dir = os.path.join(dataset_dir, 'roma')

dataset_morango_len = len(os.listdir(dataset_morango_dir))
dataset_pessego_len = len(os.listdir(dataset_pessego_dir))
dataset_roma_len = len(os.listdir(dataset_roma_dir))

print(f'Contagem de imagens de morango: {dataset_morango_len}')
print(f'Contagem de imagens de pêssego: {dataset_pessego_len}')
print(f'Contagem de imagens de romã: {dataset_roma_len}')

Contagem de imagens de morango: 306
Contagem de imagens de pêssego: 304
Contagem de imagens de romã: 311


In [3]:
# 2. Aplicando o rebalanceamento do dataset
# -> Utilizada a técnica de 'data augmentation' para gerar novas imagens de morango e pessego
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Criar um objeto ImageDataGenerator para data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,          # Faixa de rotação aleatória em graus
    width_shift_range=0.2,      # Faixa de deslocamento horizontal aleatório (como uma fração da largura total)
    height_shift_range=0.2,     # Faixa de deslocamento vertical aleatório (como uma fração da altura total)
    shear_range=0.2,            # Faixa de cisalhamento aleatório (em radianos)
    zoom_range=0.2,             # Faixa de zoom aleatório
    horizontal_flip=True,      # Inverter aleatoriamente as imagens horizontalmente (espelhamento)
    fill_mode='nearest'        # Estratégia de preenchimento usada para preencher novos pixels gerados após a rotação ou deslocamento
)

if (dataset_morango_len == 250) or (dataset_pessego_len == 250): 
    numero_imagens_aumentadas = 61
    classes_aumentadas = ['morango', 'pessego']

    for classe in classes_aumentadas:
        # Diretório da classe atual
        diretorio_classe = os.path.join(dataset_dir, classe)
        
        # Criar diretório para a classe aumentada, se não existir
        diretorio_destino_classe = os.path.join(dataset_dir, classe)
        os.makedirs(diretorio_destino_classe, exist_ok=True)
        
        # Lista de arquivos de imagem na classe atual
        imagens_classe = os.listdir(diretorio_classe)
        
        # Selecionar aleatoriamente algumas imagens existentes para data augmentation
        indices_amostra = np.random.choice(len(imagens_classe), numero_imagens_aumentadas, replace=True)
        
        # Para cada imagem de amostra selecionada
        for indice in indices_amostra:
            imagem_nome = imagens_classe[indice]
            imagem_path = os.path.join(diretorio_classe, imagem_nome)
            
            # Carregar imagem
            imagem = Image.open(imagem_path)
            imagem_array = np.array(imagem)
            imagem_array = imagem_array.reshape((1,) + imagem_array.shape)  # Reshape para (1, altura, largura, canais)
            
            # Gerar imagens aumentadas e salvar no diretório de destino
            for i, batch in enumerate(datagen.flow(imagem_array, batch_size=1)):
                if i >= 1:  # Quantidade de imagens aumentadas a serem geradas por imagem de amostra
                    break
                imagem_aumentada = batch[0].astype(np.uint8)  # Converter de volta para o formato de imagem
                nova_imagem_nome = f"{os.path.splitext(imagem_nome)[0]}_aug_{i}.jpg"
                nova_imagem_path = os.path.join(diretorio_destino_classe, nova_imagem_nome)
                nova_imagem = Image.fromarray(imagem_aumentada)
                nova_imagem.save(nova_imagem_path)

In [11]:
# 3. Definição e separação dos dados de treinamento e dados de teste
from sklearn.model_selection import train_test_split
import shutil
import random

# Diretório do projeto
projeto_dir = os.getcwd()

# Caminhos para as pastas de treinamento, validação e teste
dataset_treinamento_dir = os.path.join(projeto_dir, 'imagens_treinamento')
dataset_validacao_dir = os.path.join(projeto_dir, 'imagens_validacao')
dataset_teste_dir = os.path.join(projeto_dir, 'imagens_teste')

# Proporções para dividir os dados entre treinamento, validação e teste
proporcao_treinamento = 0.6  # 60% para treinamento
proporcao_validacao = 0.2  # 20% para validação
proporcao_teste = 0.2  # 20% para teste

# Conforme conversado com professor, utilizar a função random para gerar o random_seed
#random_seed = random.randint(1, 99)

# No entanto, estou utilizando um valor setado em 42 para ter sempre os mesmos conjuntos de dados durante os testes do código
random_seed = 42

classes = ['morango', 'pessego', 'roma']

# Função que executará a separação das imagens (utiliza algoritmo de aleatoriedade)
def split_dataset(classe):
    dataset_treinamento_classe_dir = os.path.join(dataset_treinamento_dir, classe) # Dir. destino treinamento classe
    dataset_teste_classe_dir = os.path.join(dataset_teste_dir, classe) # Dir. destino teste classe

    # Dir. origem imagens classe
    dataset_classe_dir = os.path.join(projeto_dir, 'imagens', classe)
    imagens_classe = [os.path.join(dataset_classe_dir, img) for img in os.listdir(dataset_classe_dir)] # Popular lista com imagens

    imagens_treinamento, imagens_temp = train_test_split(imagens_classe, test_size=proporcao_validacao + proporcao_teste, random_state=random_seed)
    imagens_validacao, imagens_teste = train_test_split(imagens_temp, test_size=proporcao_teste / (proporcao_validacao + proporcao_teste), random_state=random_seed)

    # Criação dos diretórios para as classes
    os.makedirs(os.path.join(dataset_treinamento_dir, classe), exist_ok=True)
    os.makedirs(os.path.join(dataset_validacao_dir, classe), exist_ok=True)
    os.makedirs(os.path.join(dataset_teste_dir, classe), exist_ok=True)
    
    imagens_classe = [os.path.join(dataset_classe_dir, img) for img in os.listdir(dataset_classe_dir)]

    # Copia as imagens para as pastas correspondentes
    for imagem in imagens_treinamento:
        shutil.copy(imagem, os.path.join(dataset_treinamento_dir, classe))
    for imagem in imagens_validacao:
        shutil.copy(imagem, os.path.join(dataset_validacao_dir, classe))
    for imagem in imagens_teste:
        shutil.copy(imagem, os.path.join(dataset_teste_dir, classe))

# Verifica se os diretórios de treinamento, validação e teste já existem
if not os.path.exists(dataset_treinamento_dir) or not os.path.exists(dataset_validacao_dir) or not os.path.exists(dataset_teste_dir):
    # Cria os diretórios
    os.makedirs(dataset_treinamento_dir, exist_ok=True)
    os.makedirs(dataset_validacao_dir, exist_ok=True)
    os.makedirs(dataset_teste_dir, exist_ok=True)

    # Para cada classe, executa a função de separação de dados
    for classe in classes:
        split_dataset(classe)

In [12]:
# Função para contar o número de imagens de cada diretório
def contar_imagens(diretorio):
    return sum([len(files) for root, dirs, files in os.walk(diretorio)])

# Contagem de imagens para treinamento e teste de cada classe
dataset_treinamento_morango_len = contar_imagens(os.path.join(dataset_treinamento_dir, 'morango'))
dataset_teste_morango_len = contar_imagens(os.path.join(dataset_teste_dir, 'morango'))
dataset_validacao_morango_len = contar_imagens(os.path.join(dataset_validacao_dir, 'morango'))
dataset_treinamento_pessego_len = contar_imagens(os.path.join(dataset_treinamento_dir, 'pessego'))
dataset_teste_pessego_len = contar_imagens(os.path.join(dataset_teste_dir, 'pessego'))
dataset_validacao_pessego_len = contar_imagens(os.path.join(dataset_validacao_dir, 'pessego'))
dataset_treinamento_roma_len = contar_imagens(os.path.join(dataset_treinamento_dir, 'roma'))
dataset_teste_roma_len = contar_imagens(os.path.join(dataset_teste_dir, 'roma'))
dataset_validacao_roma_len = contar_imagens(os.path.join(dataset_validacao_dir, 'roma'))

# Impressão das contagens
print(f'Contagem de imagens de morango para treinamento: {dataset_treinamento_morango_len}')
print(f'Contagem de imagens de morango para teste: {dataset_teste_morango_len}')
print(f'Contagem de imagens de morango para validacao: {dataset_validacao_morango_len}')
print(f'Contagem de imagens de pêssego para treinamento: {dataset_treinamento_pessego_len}')
print(f'Contagem de imagens de pêssego para teste: {dataset_teste_pessego_len}')
print(f'Contagem de imagens de pêssego para validacao: {dataset_validacao_pessego_len}')
print(f'Contagem de imagens de romã para treinamento: {dataset_treinamento_roma_len}')
print(f'Contagem de imagens de romã para teste: {dataset_teste_roma_len}')
print(f'Contagem de imagens de romã para validacao: {dataset_validacao_roma_len}')


Contagem de imagens de morango para treinamento: 183
Contagem de imagens de morango para teste: 62
Contagem de imagens de morango para validacao: 61
Contagem de imagens de pêssego para treinamento: 182
Contagem de imagens de pêssego para teste: 61
Contagem de imagens de pêssego para validacao: 61
Contagem de imagens de romã para treinamento: 186
Contagem de imagens de romã para teste: 63
Contagem de imagens de romã para validacao: 62


In [13]:
## 4. Pré-processamento das imagens
## --> Definir tamanho de entrada das minhas imagens (em px)
## --> Definir qual estratégia de conversão adotar (scaling da imagem / foco no centro da imagem ignorando periferia / recortar imagem até no limite do tamanho definido e ignorar o restante)

'''
Considerações sobre a abordagem:
O pré-processamento é feito pela função "tf.keras.preprocessing.image_dataset_from_directory". Essa função permite especificar o tamanho das imagens e aplica automaticamente a normalização durante o carregamento das imagens. Além disso, aplica embaralhamento
e carrega as imagens em lotes (batch). Por fim, as imagens são consideradas como tensores do TensorFlow.
'''

import os
import tensorflow as tf

image_width = 300
image_heigth = 300
image_size = (image_width, image_heigth)

image_color_channel = 3
image_color_channel_size = 255
image_shape = image_size + (image_color_channel,)

batch_size = 32 # Valor que vou puxar do dataset por vez
epoch = 20 # Quantidade de vezes que vou percorrer meu dataset inteiro
learning_rate = 0.0001 # Taxa de aprendizagem

classes = ['morango', 'pessego', 'roma']

print(dataset_treinamento_dir)
print(dataset_teste_dir)

data_set_treinamento = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_treinamento_dir,
    image_size = image_size,
    batch_size = batch_size,
    shuffle = True, # Embaralhamento
    label_mode='categorical' # Carrega os dados em formato one-hot
)

data_set_validacao = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_validacao_dir,
    image_size = image_size,
    batch_size = batch_size,
    shuffle = False,
    label_mode='categorical'
)

data_set_teste = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_teste_dir,
    image_size = image_size,
    batch_size = batch_size,
    shuffle = False, # Não é necessário embaralhar os dados de teste
    label_mode='categorical'
)

/home/matheus/Documentos/GitHub/esw-pin3-projeto/modelo_1/imagens_treinamento
/home/matheus/Documentos/GitHub/esw-pin3-projeto/modelo_1/imagens_teste
Found 551 files belonging to 3 classes.


Found 184 files belonging to 3 classes.
Found 186 files belonging to 3 classes.


In [14]:
# 5. Definição da arquitetura da rede neural do modelo

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

def create_model(learning_rate, optimizer):
  """
  Cria um modelo de rede neural convolucional com parâmetros específicos.

  Args:
      learning_rate (float): A taxa de aprendizado para o otimizador especificado.
      optimizer (str): O nome do otimizador a ser utilizado ('adam', 'sgd', ou 'rmsprop').

  Returns:
      tf.keras.models.Sequential: O modelo criado.
  """

  # Define o otimizador com base no parâmetro passado
  if optimizer == 'adam':
      optimizer = Adam(learning_rate)
  elif optimizer == 'sgd':
      optimizer = SGD(learning_rate)
  elif optimizer == 'rmsprop':
      optimizer = RMSprop(learning_rate)
  else:
      raise ValueError("Otimizador inválido. Escolha entre 'adam', 'sgd' ou 'rmsprop'.")

  # Arquitetura da rede do modelo implementado:
  arc_model = Sequential([
    # Definição do tipo de entrada: imagens de 300x300 pixels com 3 canais de cores (RGB) 
    Input(shape=(300,300,3)),

    # Primeira camada: convolucional com 32 filtros de tamanho 3x3, utilizando a função de ativação ReLU
    # Cada filtro aprende padrões diferentes de cada imagem.
    # Tamanho 3x3 significa que o filtro vai processar com fragmentos de 3x3 pixels
    # Função de ativação: introduz não linearidade na rede.
    Conv2D(32, (3,3), activation='relu'),

    # Camada de normalização de batch, para normalizar a ativação da camada anterior
    # Ajuda a estabilizar e normalizar as ativações durante o treinamento.
    BatchNormalization(),

    # Camada de MaxPooling, realiza downsampling (compressão) da entrada
    # Processa em tamanho de 2x2 pixels 
    MaxPooling2D((2,2)),

    # Mais uma camada Conv2D, com 64 filtros de tamanho 3x3, com função de ativação ReLU
    Conv2D(64, (3,3), activation='relu'),

    # Outra camada de normalização de batch, para normalizar a ativação da camada anterior
    BatchNormalization(),

    # Outra Camda de MaxPooling
    MaxPooling2D((2,2)),

    # Camada de achatamento (Flatten) para transformar os mapas de características 2D em um vetor 1D
    # Esta camada prepara os dados para entrada nas camadas densas (abaixo)
    Flatten(),
    
    # Camada densa (totalmente conectada) com 128 neurônios e função de ativação ReLU
    Dense(128, activation='relu'),
    
    # Mias uma camada de normalização de batch, para normalizar a ativação da camada anterior
    BatchNormalization(),

    # Mais uma camada densa com 64 neurônios e função de ativação ReLU
    Dense(64, activation='relu'),
    
    # Camada de Dropout para prevenir overfitting, desativando aleatoriamente 50% dos neurônios
    Dropout(0.5),
    
    # Camada de saída com 3 neurônios (um para cada classe) e função de ativação softmax para a classificação multiclasse
    # A saída é uma distribuição de probabilidade sobre as classes
    Dense(3, activation='softmax')
  ])

  # Compilação do modelo
  # Aqui, utilizamos a função de perda conforme espeficicado no documento do projeto
  # Função de perda: crossentropy, que irá calcular a diferença entre as previsões realizadas pelo modelo e os rótulos verdadeiros associados aos dados de treinamento.
  arc_model.compile(optimizer=optimizer, loss=CategoricalCrossentropy(), metrics=['accuracy'])


# Além disso, testaremos 3 tipos de funções de otimização:

# Modelo com função de otimização Adam
model_adam = create_model(optimizer='adam', learning_rate=0.0001)

# Modelo com função de otimização SGD
model_sgd = create_model(optimizer='sgd', learning_rate=0.0001)

# Modelo com função de otimização RMSprop
model_rmsprop = create_model(optimizer='sgd', learning_rate=0.0001)

In [19]:
# 6. Treinamento do modelo e algumas validações com accuray e confusion matrix

# Declaração da função de callback que será chamada no fim do processamento de cada época durante o treinamento
# Calcula a matriz de confusão e teste de acurácia
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np 

class MetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, dados_validacao):
        self.dados_validacao = dados_validacao

    # Vai ser chamado no final de cada época durante o treinamento
    def on_epoch_end(self, epoch, logs=None):
        y_true = []
        y_pred = []
        
        # Iterando sobre os dados para obter os rótulos verdadeiros e preditos
        for x_batch, y_batch in self.dados_validacao:
            y_true.extend(np.argmax(y_batch, axis=1))

            y_pred_batch = np.argmax(self.model.predict(x_batch), axis=1)
            y_pred.extend(y_pred_batch)

        # Calculando a matriz de confusão
        cm = confusion_matrix(y_true=y_true, y_pred=y_pred)

        # Calculando a acurácia
        accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)

        # Impressão dos resultados
        print("Matriz de confusão: ")
        print(cm)
        print('Acurácia da epoch: ', accuracy)


# Instância das métricas (acurácia e matriz de confusão)
metrics_callback = MetricsCallback(data_set_validacao)


# Treinamento do modelo com função de otimização Adam
# É passado o dataset de treinamento, a quantidade de épocas (quantas vezes vai percorrer o dataset) e o dataset de validação
model_adam.fit(
    data_set_treinamento,
    epochs=epoch,
    validation_data=data_set_validacao,
    callbacks=[metrics_callback]
)

# Teste de acurácia com o dataset de testes
teste_perca, teste_acuracia = model_adam.evaluate(data_set_teste)
print('Teste acurácia com dataset de testes: ', teste_acuracia)

'''

Epochs: Cada linha corresponde a uma época de treinamento. Uma época é uma iteração completa sobre todo o conjunto de dados de treinamento.

Batches por Época: Aqui, o número 23/23 indica que há 23 batches (ou lotes) de dados sendo processados em cada época. Isso pode variar dependendo do tamanho do conjunto de treinamento e do tamanho do lote (batch_size). Por exemplo, se você tem 700 amostras de treinamento e está usando um tamanho de lote de 32, então cada época terá 22 batches (700/32 = 21.875).

Tempo por Época: O tempo indicado representa o tempo gasto para completar uma época. Isso pode variar dependendo da complexidade do modelo, tamanho do conjunto de dados e recursos do hardware.

Accuracy (Acurácia) e Loss (Perda): A acurácia e a perda durante o treinamento e validação são mostradas para cada época. A acurácia indica a proporção de previsões corretas em relação ao total de exemplos, enquanto a perda (loss) é uma medida do quão bem o modelo está performando durante o treinamento, sendo minimizada ao longo das épocas.

Val_accuracy e Val_loss: Estes são os valores de acurácia e perda calculados no conjunto de dados de validação, que é usado para avaliar o desempenho do modelo em dados que não foram vistos durante o treinamento. É importante monitorar essas métricas para evitar overfitting e garantir que o modelo esteja generalizando bem para novos dados.

Test accuracy e Test loss: Após o treinamento, o modelo é avaliado no conjunto de dados de teste (geralmente separado do conjunto de validação). Estas métricas fornecem uma avaliação final do desempenho do modelo em dados completamente novos e não vistos durante o treinamento.

Acurácia: No final do treinamento, a acurácia nos dados de validação foi de aproximadamente 84.41%. Isso significa que o modelo classificou corretamente cerca de 84.41% das amostras nos dados de validação.

Perda: A perda nos dados de validação foi de aproximadamente 0.4956. A perda é uma medida de quão boa ou ruim é a previsão do modelo para uma única amostra; valores mais baixos são melhores.

Tendência ao longo das épocas: A acurácia e a perda parecem estar se estabilizando ao longo das épocas, o que sugere que o modelo está convergindo para uma solução.
'''


Epoch 1/20


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/stepep - accuracy: 0.9291 - loss: 0.239
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Confusion Matrix: 
[[60  0  1]
 [39 21  1]
 [55  2  5]]
Accuracy:  0.4673913043478261
18/18 ━━━━━━━━━━━━━━━━━━━━ 120s 7s/step - accuracy: 0.9285 - loss: 0.2399 - val_accuracy: 0.4674 - val_loss: 1.6694
Epoch 2/20


2024-04-21 00:27:01.176675: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/stepep - accuracy: 0.9483 - loss: 0.200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 880ms/step
Confusion Matrix: 
[[59  0  2]
 [18 32 11]
 [25  5 32]]
Accuracy:  0.6684782608695652
18/18 ━━━━━━━━━━━━━━━━━━━━ 121s 7s/step - accuracy: 0.9479 - loss: 0.1999 - val_accuracy: 0.6685 - val_loss: 0.7523
Epoch 3/20


2024-04-21 00:29:01.808134: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/stepep - accuracy: 0.9984 - loss: 0.017
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 895ms/step
Confusion Matrix: 
[[52  2  7]
 [ 0 53  8]
 [ 5  7 50]]
Accuracy:  0.842391304347826
18/18 ━━━━━━━━━━━━━━━━━━━━ 115s 6s/step - accuracy: 0.9984 - loss: 0.0172 - val_accuracy: 0.8424 - val_loss: 0.4328
Epoch 20/20


2024-04-21 01:02:27.091712: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/stepep - accuracy: 0.9855 - loss: 0.031
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 881ms/step
Confusion Matrix: 
[[44  4 13]
 [ 0 44 17]
 [ 2  5 55]]
Accuracy:  0.7771739130434783
18/18 ━━━━━━━━━━━━━━━━━━━━ 116s 6s/step - accuracy: 0.9859 - loss: 0.0315 - val_accuracy: 0.7772 - val_loss: 0.5368


2024-04-21 01:04:23.444508: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


'\nEpochs: Cada linha corresponde a uma época de treinamento. Uma época é uma iteração completa sobre todo o conjunto de dados de treinamento.\n\nBatches por Época: Aqui, o número 23/23 indica que há 23 batches (ou lotes) de dados sendo processados em cada época. Isso pode variar dependendo do tamanho do conjunto de treinamento e do tamanho do lote (batch_size). Por exemplo, se você tem 700 amostras de treinamento e está usando um tamanho de lote de 32, então cada época terá 22 batches (700/32 = 21.875).\n\nTempo por Época: O tempo indicado representa o tempo gasto para completar uma época. Isso pode variar dependendo da complexidade do modelo, tamanho do conjunto de dados e recursos do hardware.\n\nAccuracy (Acurácia) e Loss (Perda): A acurácia e a perda durante o treinamento e validação são mostradas para cada época. A acurácia indica a proporção de previsões corretas em relação ao total de exemplos, enquanto a perda (loss) é uma medida do quão bem o modelo está performando durante o

In [32]:
from keras_tuner.tuners import GridSearch
from keras_tuner.engine.hyperparameters import HyperParameters

# Define os hiperparâmetros
hyperparameters = HyperParameters()
hyperparameters.Int('epochs', min_value=10, max_value=50, step=10)

# Define o tuner GridSearch
tuner = GridSearch(
    model_adam,            # Função que constrói o modelo
    hyperparameters,        # Hiperparâmetros a serem otimizados
    max_trials=9,           # Número total de tentativas de modelo (combinatórias)
    directory='grid_search', # Diretório para salvar resultados do tuner
    project_name='grid_search_example' # Nome do projeto
)

# Realiza a busca
tuner.search(data_set_treinamento, validation_data=data_set_teste)

TypeError: GridSearch.__init__() got multiple values for argument 'objective'